In [ ]:
from MultiActorEnv import MultiActorEnv
from FCNN import FCNN

import settings
from utility import MovingAveragePlot, RoutePlot, get_optimal_movs, PlayOnce
from run import init_seed, init_weights, TrainQlearning

In [ ]:
model_settings = {
    'actor_number' : 4,
    'height' : 10,
    'width' : 10,
    'obs_count' : 5,
    'random_seed' : 100
}

In [ ]:
env = MultiActorEnv(**model_settings)
print(env.grid)
env.display()

In [ ]:
input_dim = 4 * model_settings['height'] * model_settings['width'] + len(env.actions)
model = FCNN(input_dim)
config = settings.Config(epsilon=0.5, 
                         epsilon_low=0.1, 
                         epsilon_step=0.05, 
                         max_play_length=50,
                         epochs=1000, 
                         gamma=0.99)

In [ ]:
rewards = TrainQlearning(model, env, config)

In [ ]:
MovingAveragePlot(rewards, 100)

In [ ]:
import numpy as np
import random
env.reset()
print(env.grid)
env.display()
print(env.get_actor_number())
print("-------------------------------------------------------------")
counter = 0
max_play_length = 50
model.eval()
while (counter <= max_play_length and env.get_actor_number() > 0):
    # find Q(s_{t},a) for all actions
    
    counter += 1
    actor_number = env.get_actor_number()
    if (actor_number==0): break
    
    _, choices = get_optimal_movs(env, model)
    print(choices)
    env.step(choices)
    print(env.grid)
    env.display()
    env.remove_dones()
    print(env.get_actor_number())
    print("-------------------------------------------------------------")

In [ ]:
PlayOnce(env, model, config)